#0.環境構築

##0-1.まずは以下のサイトよりDeepInfraのアカウントを作成してください。

https://deepinfra.com/

##0-2.以下よりAPI KEYを取得してセルにセットしてください。

https://deepinfra.com/dash/api_keys

In [ ]:
DEEPINFRA_API_KEY = "JtsJG3gUjJ8TlaRJZUgFIkKYoQfoAbNS"

##0-3.Google Driveをマウント

エクセルファイルの保存先パスを設定してください。

In [ ]:
G_FOLDER = "/content/drive/MyDrive/geniac"

##0-4.以下のコードで必要なライブラリのインストールと、APIのテスト実行を行います。

In [ ]:
# Assume openai>=1.0.0
from openai import OpenAI

# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=DEEPINFRA_API_KEY,
    base_url="https://api.deepinfra.com/v1/openai",
)

In [ ]:
chat_completion = openai.chat.completions.create(
    model="mistralai/Mixtral-8x22B-Instruct-v0.1",
    messages=[
        {"role": "system", "content": "あなたは日本語で回答するAIボットです。"},
        {
            "role": "user",
            "content": "以下の文章が文法的に正しいかどうかを評価してください。文法的に正しい場合には1を、誤りを含んでいる場合には0を選択してください。  本件に関しましては、貴社のご要望を踏まえて最適なソリューションをご提案させていただきます次第でございます。",
        },
    ],
)

print(chat_completion.choices[0].message.content)

 1


#1.データ生成処理の実行

##1-1.プロンプト準備

言語理解でのプロンプト例

In [1]:
PROMPT_TEMPLATE = """
以下のタスク指示例を参考に、10種類の多様なinputとoutputを考えてください。これらのタスク指示はGPTモデルに与えられ、その指示を完了するためのGPTモデルを評価します。

要件は以下の通りです：
1. 同じ動詞や形容詞などを繰り返さないようにして、多様性を最大限に引き出してください。
2. 出力は指示に対する適切な応答であるべきです。出力は1か0であることを確認してください。
3. 生成するinputとoutputは全て必ず日本語で生成してください。
4. 多様なシーンでの情報抽出能力を問うことができる問題を作成してください。
5. [市場調査の実施, 財務報告の作成, 採用面接の実施, プロジェクト進捗の管理, カスタマーサポートの対応, 営業プレゼンテーションの実施, 新製品のアイデア会議, 在庫管理の最適化, チームミーティングのリーダーシップ, データ分析による戦略立案]の中からテーマを選び、タスクを作成してください。
6. 回答フォーマット例に記載のJSON形式で回答してください。

タスク指示例：
$FewShotExamples

回答フォーマット例(JSON):
[
  {
    "id": 1,
    "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
    "context": "<情報>",
    "question": "<設問>",
    "output": "<回答>"
  },
  {
    "id": 2,
    "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
    "context": "<情報>",
    "question": "<設問>",
    "output": "<回答>"
  },
  {
    "id": 3,
    "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
    "context": "<情報>",
    "question": "<設問>",
    "output": "<回答>"
  }
]
"""

In [8]:
FEWSHOT_EXAMPLES = [
    {
        "id": 1,
        "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
        "context": "各位 いつもお世話になっております。山田商事の田中です。平素よりご愛顧いただきまして誠にありがとうございます。本メールは、弊社の新製品発表会に関するご案内です。発表会は来月の15日に開催され、最新の製品ラインナップを紹介いたします。参加ご希望の方は、以下のリンクより事前登録をお願いいたします。",
        "question": "このメールの主な目的は何ですか。",
        "output": "新製品発表会の案内",
    },
    {
        "id": 2,
        "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
        "context": "お客様各位 平素よりお世話になっております。ABC株式会社の佐藤です。この度、弊社は新しいサポートサービスを開始いたしました。開始を記念して、6月1日にウェビナーを開催いたします。ウェビナーでは、新サービスの詳細と利用方法についてご説明いたします。ぜひご参加ください。",
        "question": "イベントの日付はいつですか。",
        "output": "6月1日",
    },
    {
        "id": 3,
        "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
        "context": "お世話になっております。XYZ株式会社の山田です。弊社製品をご利用いただき誠にありがとうございます。現在、製品改善のためのアンケートを実施しております。ご協力いただける場合は、以下のリンクからご回答ください。回答期限は今月末までです。ご協力よろしくお願いいたします。",
        "question": "アンケートの回答期限はいつですか。",
        "output": "今月末",
    },
    {
        "id": 4,
        "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
        "context": "株式会社ABCの佐藤です。お世話になっております。この度、新製品の発売に伴い、プレスリリースを送付いたします。詳細につきましては、添付ファイルをご覧ください。どうぞよろしくお願いいたします。",
        "question": "このメールの目的は何ですか。",
        "output": "プレスリリースの送付",
    },
    {
        "id": 5,
        "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
        "context": "お世話になっております。株式会社XYZの高橋です。平素よりご利用いただきありがとうございます。今回、弊社のサポートプランの更新時期が近づいております。更新の手続きを希望される場合は、今週金曜日までにご連絡ください。よろしくお願いいたします。",
        "question": "締め切り日はいつですか。",
        "output": "今週金曜日",
    },
    {
        "id": 6,
        "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
        "context": "お世話になっております。ABC株式会社の中村です。平素よりお世話になっております。新製品の発表に伴い、質問がございましたら下記のメールアドレスまでご連絡ください。info@abc.co.jp 何卒よろしくお願いいたします。",
        "question": "問い合わせ先はどこですか。",
        "output": "info@abc.co.jp",
    },
    {
        "id": 7,
        "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
        "context": "お世話になっております。株式会社DEFの山本です。次回のチームミーティングは来週の水曜日に開催予定です。詳細は後日お知らせいたしますので、スケジュールを確保しておいてください。",
        "question": "次の会議の予定はいつですか。",
        "output": "来週の水曜日",
    },
    {
        "id": 8,
        "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
        "context": "お世話になっております。プロジェクトマネージャーの田中です。現在進行中のプロジェクトのデッドラインが来月末に迫っております。各担当者は、進捗状況を確認し、遅れがないようにご対応ください。よろしくお願いいたします。",
        "question": "プロジェクトのデッドラインはいつですか。",
        "output": "来月末",
    },
    {
        "id": 9,
        "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
        "context": "お世話になっております。株式会社GHIの鈴木です。この度、新しいカスタマーサポートシステムの導入に伴い、トレーニングセッションを実施いたします。詳細は以下のリンクからご確認いただけます。ご参加お待ちしております。",
        "question": "このメールの目的は何ですか。",
        "output": "トレーニングセッションの案内",
    },
    {
        "id": 10,
        "instruction": "質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
        "context": "お世話になっております。株式会社JKLの井上です。弊社は、次の新製品を来月の10日に発表予定です。発表会の詳細は追ってご連絡いたしますので、日程を確保していただければ幸いです。",
        "question": "新製品の発表日はいつですか。",
        "output": "来月の10日",
    },
]

In [9]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(
    FewShotExamples=FEWSHOT_EXAMPLES
)
print(formatted_prompt_template)


以下のタスク指示例を参考に、10種類の多様なinputとoutputを考えてください。これらのタスク指示はGPTモデルに与えられ、その指示を完了するためのGPTモデルを評価します。

要件は以下の通りです：
1. 同じ動詞や形容詞などを繰り返さないようにして、多様性を最大限に引き出してください。
2. 出力は指示に対する適切な応答であるべきです。出力は1か0であることを確認してください。
3. 生成するinputとoutputは全て必ず日本語で生成してください。
4. 多様なシーンでの情報抽出能力を問うことができる問題を作成してください。
5. [市場調査の実施, 財務報告の作成, 採用面接の実施, プロジェクト進捗の管理, カスタマーサポートの対応, 営業プレゼンテーションの実施, 新製品のアイデア会議, 在庫管理の最適化, チームミーティングのリーダーシップ, データ分析による戦略立案]の中からテーマを選び、タスクを作成してください。
6. 回答フォーマット例に記載のJSON形式で回答してください。

タスク指示例：
[{'id': 1, 'instruction': '質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。', 'context': '各位 いつもお世話になっております。山田商事の田中です。平素よりご愛顧いただきまして誠にありがとうございます。本メールは、弊社の新製品発表会に関するご案内です。発表会は来月の15日に開催され、最新の製品ラインナップを紹介いたします。参加ご希望の方は、以下のリンクより事前登録をお願いいたします。', 'question': 'このメールの主な目的は何ですか。', 'output': '新製品発表会の案内'}, {'id': 2, 'instruction': '質問に対する回答を文章から一言で抽出してください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。', 'context': 'お客様各位 平素よりお世話になっております。ABC株式会社の佐藤です。この度、弊社は新しいサポートサービスを開始いたしました。開始を記念して、6月1日にウェビナーを開催いたします。ウェビナーでは、新サービスの詳細と利用方法についてご

##1-2.データ生成処理の実行

パラメーターをセット

In [ ]:
TEMPERATURE = 1  # temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
MAX_TOKENS = 10000
LOOP_NUM = 1  # 1回あたり10件データを生成

In [ ]:
def run_model(PROMPT_TEMPLATE, TEMPERATURE=1, MAX_TOKENS=10000):

    chat_completion = openai.chat.completions.create(
        model="mistralai/Mixtral-8x22B-Instruct-v0.1",
        messages=[
            {
                "role": "system",
                "content": "あなたは日本語の学習データセットを生成する専門家です。",
            },
            {"role": "user", "content": PROMPT_TEMPLATE},
        ],
        temperature=TEMPERATURE,  # temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
        top_p=0.9,  # 0 < top_p ≤ 1 Sample from the set of tokens with highest probability such that sum of probabilies is higher than p. Lower values focus on the most probable tokens.Higher values sample more low-probability tokens
        max_tokens=MAX_TOKENS,
        frequency_penalty=1.3,  # -2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on how many times they appear in the text so far, increasing the model's likelihood to talk about new topics.
        presence_penalty=1.3,  # -2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.
    )

    output = chat_completion.choices[0].message.content

    return output

In [ ]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(
    FewShotExamples=FEWSHOT_EXAMPLES
)

for i in range(LOOP_NUM):

    ### JSON形式で出力が出てこないケースがあるため、3回までリトライ ###
    for retry_i in range(3):

        ### LLMによるデータ生成処理 ###
        output = run_model(formatted_prompt_template, TEMPERATURE, MAX_TOKENS)
        print(output)

        try:
            # output = json.loads(output)
            df_output = pd.read_json(output)
            df_list.append(df_output)
            print(f"Success parsing JSON")
            break
        except Exception as e:
            print(e)
            print(f"Error parsing JSON: {retry_i}")
            continue


df_final = pd.concat(df_list)

df_final.head()

 [
  {
    "id": 1,
    "instruction": "この文章から、財務報告書の完成日を一言で答えてください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
    "context": "この度は、弊社の売上高が前年度同期比で20%増加しました。この成績はすべての従業員に感謝します。なお、今月末までに財務報告書を完成させていただきたいと考えております。",
    "question": "財務報告書の完成日はいつですか。",
    "output": "今月末"
  },
  {
    "id": 2,
    "instruction": "この文章から、採用面接を行う企業の名前を一言で答えてください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
    "context": "ABC会社では、次回の採用面接は来週月曜日に行われます。面接に通う申し込みは、メールまたは弊社のHPから行っていただけます。",
    "question": "採用面接を行う企業の名前は何ですか。",
    "output": "ABC会社"
  },
  {
    "id": 3,
    "instruction": "この文章から、新製品開発のアイデアを一言で答えてください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
    "context": "XYZ社では、次回の会議で、新製品開発についてアイデアを募集します。例えば、自動運転車の話などがあります。",
    "question": "新製品開発のアイデアは何ですか。",
    "output": "自動運転車"
  },
  {
    "id": 4,
    "instruction": "この文章から、データ分析による戦略立案の結果を一言で答えてください。回答は名詞で答えてください。それ以外には何も含めないことを厳守してください。",
    "context": "データ分析による結果、弊社は競合他社に比べて成長速度が速かったことがわかりました。そのため、今後は高速成長を継続するための戦略立案に取り組むことになりました。",
    "question": "データ分

/var/folders/bs/wbh7rgdj2fngtr9pnqy4n_mm0000gq/T/ipykernel_34980/2650223231.py:24: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_output = pd.read_json(output)


,id,instruction,context,question,output
0,1,この文章から、財務報告書の完成日を一言で答えてください。回答は名詞で答えてください。それ以外...,この度は、弊社の売上高が前年度同期比で20%増加しました。この成績はすべての従業員に感謝しま...,財務報告書の完成日はいつですか。,今月末
1,2,この文章から、採用面接を行う企業の名前を一言で答えてください。回答は名詞で答えてください。そ...,ABC会社では、次回の採用面接は来週月曜日に行われます。面接に通う申し込みは、メールまたは弊...,採用面接を行う企業の名前は何ですか。,ABC会社
2,3,この文章から、新製品開発のアイデアを一言で答えてください。回答は名詞で答えてください。それ以...,XYZ社では、次回の会議で、新製品開発についてアイデアを募集します。例えば、自動運転車の話な...,新製品開発のアイデアは何ですか。,自動運転車
3,4,この文章から、データ分析による戦略立案の結果を一言で答えてください。回答は名詞で答えてくださ...,データ分析による結果、弊社は競合他社に比べて成長速度が速かったことがわかりました。そのため、...,データ分析による戦略立案の結果は何ですか。,高速成長
4,5,この文章から、市場調査を行う企業の名前を一言で答えてください。回答は名詞で答えてください。そ...,DEF会社では、次期市場調査を行います。調査対象は新製品に関する市場動向となります。,市場調査を行う企業の名前は何ですか。,DEF会社


##1-3.データの保存

In [ ]:
df_final = pd.concat(df_list)

In [ ]:
import datetime

# Get the current datetime
now = datetime.datetime.now()

# Format the datetime as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with the timestamp
filename = f"/v1_generated_data_{timestamp}.xlsx"

# Save the DataFrame to an Excel file with the timestamped filename
df_final.to_excel("output" + filename, index=False)

print(f"File saved as: {filename}")

File saved as: /v1_generated_data_2024-05-17_12-13-31.xlsx
